In [1]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import os
import pandas as pd
import shutil
import sqlalchemy as sa

import database as db
from images import get_list_of_images_invalid
from models import County, DataTrustedIdentifier, Image
from unaccent import unaccent

In [3]:
engine, session=db.connect()
engine.echo=False

2023-03-29 13:19:36,816 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-03-29 13:19:36,816 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-29 13:19:36,817 INFO sqlalchemy.engine.Engine select current_schema()
2023-03-29 13:19:36,818 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-29 13:19:36,819 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-03-29 13:19:36,819 INFO sqlalchemy.engine.Engine [raw sql] {}


In [4]:
query = session.query(County).distinct().all()

list_uf = [unaccent(sa.func.lower(q.uf)) for q in query]
list_state = [unaccent(sa.func.lower(q.state)) for q in query]
list_county = [unaccent(sa.func.lower(q.county)) for q in query]

uf_unaccented_lower = unaccent(sa.func.lower(DataTrustedIdentifier.state_province)).in_(list_uf)
state_unaccented_lower = unaccent(sa.func.lower(DataTrustedIdentifier.state_province)).in_(list_state)
county_unaccented_lower = unaccent(sa.func.lower(DataTrustedIdentifier.county)).in_(list_county)

In [5]:
list_minimum_image=[5]#, 10, 20]
list_color=['rgb']#, 'grayscale']
list_image_size=['256']#, '400', '512']
path_out = '/home/xandao/Imagens/br_dataset'
list_images_invalid=get_list_of_images_invalid()
list_images_invalid=list_images_invalid['barcode']
list_images_invalid

['INPA0248526',
 'INPA0248523',
 'INPA0248528',
 'NY01421926_01',
 'NY01421575_01',
 'HUFSJ001689_v00',
 'HUFSJ001133_v00',
 'HUFSJ002198_v00',
 'HUFSJ003255_v00',
 'HVASF000487_v01',
 'INPA0019084_nd',
 'INPA0022379_nd',
 'INPA0032742_nd',
 'INPA0023115',
 'NL-U1484137',
 'INPA0012286',
 'INPA0146998']

In [8]:
def copy_images(list_level, list_path_images_final, out):
    if not os.path.exists(out):
        os.makedirs(out)

    for i, ff in enumerate(zip(list_level, list_path_images_final), start=1):
        level = ff[0]
        list_images = ff[1]

        out_level = os.path.join(out, 'f%d' % i)
        if not os.path.exists(out_level):
            os.makedirs(out_level)

        for i, image in enumerate(list_images, start=1):
            print('copy %d of %d' % (i, len(list_images)))
            shutil.copy(image, out_level)


for color in list_color:
    for image_size in list_image_size:
        for minimum_image in list_minimum_image:
            list_level = []
            list_path_images_final = []
            list_seq = []
            list_count_samples = []

            out=os.path.join(path_out, color.upper(), image_size, str(minimum_image))

            condition = sa.and_(DataTrustedIdentifier.country_trusted == 'Brasil',
                                DataTrustedIdentifier.specific_epithet_trusted.is_not(None),
                                sa.or_(uf_unaccented_lower, state_unaccented_lower))

            columns = [DataTrustedIdentifier.specific_epithet_trusted,
                       sa.func.array_agg(DataTrustedIdentifier.seq).label('list_seq')]
            query = session.query(*columns) \
                .filter(condition) \
                .distinct() \
                .group_by(DataTrustedIdentifier.specific_epithet_trusted) \
                .order_by(DataTrustedIdentifier.specific_epithet_trusted) \
                .having(sa.func.count(DataTrustedIdentifier.specific_epithet_trusted) >= minimum_image) \
                .all()

            for i, q in enumerate(query):
                print('color: %s image_size: %s minimum_image: %d (%d of %d)' % (color.upper(), image_size, minimum_image, i, len(query)))
                qzao = session.query(Image.seq_id, sa.func.array_agg(Image.path).label('list_path')) \
                    .filter(sa.and_(Image.seq_id.in_(q.list_seq),
                                    Image.color_mode.__eq__(color.upper()),
                                    Image.height.__eq__(image_size),
                                    Image.width.__eq__(image_size)),
                                    sa.not_(Image.filename.in_(list_images_invalid)),
                                    sa.or_(sa.not_(Image.filename.ilike('_v0')),
                                            sa.not_(Image.filename.ilike('_nd'))))\
                    .group_by(Image.seq_id) \
                    .all()

                list_one_image_per_seq = []
                for qzinho in qzao:
                    list_one_image_per_seq.append(sorted(qzinho.list_path)[0])

                if len(list_one_image_per_seq) >= minimum_image:
                    list_level.append(q.specific_epithet_trusted)
                    list_path_images_final.append(list_one_image_per_seq)
                    list_count_samples.append(len(list_one_image_per_seq))

            copy_images(list_level, list_path_images_final, out)

color: RGB image_size: 256 minimum_image: 5 (0 of 246)
color: RGB image_size: 256 minimum_image: 5 (1 of 246)
color: RGB image_size: 256 minimum_image: 5 (2 of 246)
color: RGB image_size: 256 minimum_image: 5 (3 of 246)
color: RGB image_size: 256 minimum_image: 5 (4 of 246)
color: RGB image_size: 256 minimum_image: 5 (5 of 246)
color: RGB image_size: 256 minimum_image: 5 (6 of 246)
color: RGB image_size: 256 minimum_image: 5 (7 of 246)
color: RGB image_size: 256 minimum_image: 5 (8 of 246)
color: RGB image_size: 256 minimum_image: 5 (9 of 246)
color: RGB image_size: 256 minimum_image: 5 (10 of 246)
color: RGB image_size: 256 minimum_image: 5 (11 of 246)
color: RGB image_size: 256 minimum_image: 5 (12 of 246)
color: RGB image_size: 256 minimum_image: 5 (13 of 246)
color: RGB image_size: 256 minimum_image: 5 (14 of 246)
color: RGB image_size: 256 minimum_image: 5 (15 of 246)
color: RGB image_size: 256 minimum_image: 5 (16 of 246)
color: RGB image_size: 256 minimum_image: 5 (17 of 246)
co

In [ ]:
df = pd.DataFrame({
    'levels': list_level,
    'paths': list_path_images_final,
    'count': list_count_samples,
#    'seq': list_seq,
})
df['count'].sum()

In [ ]:
session.commit()
session.flush()